# S-parameter Correction and Manipulation
Examples of how to apply a 12-term and 8-term correction model to s-parameter data. In addition the transformation between S and T matrices. 

In [1]:
from pyMeasure import *
import os
import numpy as np

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [2]:
# import an s-parameter file to play with
STATISTICAL_TEST_DIRECTORY=r'C:\Users\sandersa\PyCharm Projects\Jupyter-Notebooks\StatistiCAL_Files\StatistiCAL_Tests'
os.chdir(STATISTICAL_TEST_DIRECTORY)
two_port=S2PV1("DY202.txt")
two_port.show()

In [3]:
# Now create a numpy matrix of sparameters to do T-matrix conversion, should be added to S2PV1 model
matrix_form=[[row[0],np.matrix(row[1:]).reshape((2,2))] for row in two_port.sparameter_complex]

In [4]:
# Convert to T_matrix form, also should be added to S2PV1
t_matrix=[]
for row in matrix_form:
    frequency=row[0]
    m=row[1]
    T11=-np.linalg.det(m)/m[1,0]
    T12=m[0,0]/m[1,0]
    T21=-m[1,1]/m[1,0]
    T22=1/m[1,0]
    t_matrix.append([frequency,np.matrix([[T11,T21],[T21,T22]])])

In [5]:
def S_to_T(S_list):
    """Converts S-parameters into a T Matrix. Input form should be in frequency, np.matrix([[S11,S12],[S21,S22]])
    format. Returns a list in [frequency, np.matrix] format """
    t_matrix=[]
    for row in S_list:
        frequency=row[0]
        m=row[1]
        T11=-np.linalg.det(m)/m[1,0]
        T12=m[0,0]/m[1,0]
        T21=-m[1,1]/m[1,0]
        T22=1/m[1,0]
        t_matrix.append([frequency,np.matrix([[T11,T12],[T21,T22]])])
    return t_matrix

def T_to_S(T_list):
    """Converts T Matrix into S parameters. Input form should be in frequency, np.matrix([[T11,T12],[T21,T22]])
    format. Returns a list in [frequency, np.matrix] format."""
    S_list=[]
    for row in T_list:
        frequency=row[0]
        m=row[1]
        S11=m[0,1]/m[1,1]
        S12=np.linalg.det(m)/m[1,1]
        S21=1/m[1,1]
        S22=-m[1,0]/m[1,1]
        S_list.append([frequency,np.matrix([[S11,S12],[S21,S22]])])
    return S_list

In [6]:
# now we try it on the matrix form
t_matrix=S_to_T(matrix_form)
s_again=T_to_S(t_matrix)

# lets compare the results
print("Before conversion to T the first row is {0}".format(matrix_form[0]))
print("After the conversion to T the first row is {0}".format(t_matrix[0]))
print("Converting back to S the first row is {0}".format(s_again[0]))
print("The assertion S before is equal to S after is {0} ".format(matrix_form[0][1].all()==s_again[0][1].all()))

Before conversion to T the first row is [0.2, matrix([[ 0.0014908-0.0053344j, -0.0363140+0.0134182j],
        [-0.0208168-0.0321827j,  0.0092726+0.0106483j]])]
After the conversion to T the first row is [0.2, matrix([[ -0.03604906 +1.13950410e-02j,   0.09573550 +1.08247811e-01j],
        [  0.36466513 -5.22466703e-02j, -14.17009810 +2.19069221e+01j]])]
Converting back to S the first row is [0.2, matrix([[ 0.0014908-0.0053344j, -0.0363140+0.0134182j],
        [-0.0208168-0.0321827j,  0.0092726+0.0106483j]])]
The assertion S before is equal to S after is True 


In [7]:
def correct_sparameters(sparameters_complex,twelve_term_correction):
    """Applies the twelve term correction to sparameters and returns a new sparameter list."""
    if len(sparameters_complex) != len(twelve_term_correction):
        raise TypeError("s parameter and twelve term correction must be the same length")
    s_parameter_out=[]
    for index,row in enumerate(sparameters_complex):
        frequency=row[0]
        Sm=np.matrix(row[1:]).reshape((2,2))
        [frequency,Edf,Esf,Erf,Exf,Elf,Etf,Edr,Esr,Err,Exr,Elr,Etr]=twelve_term_correction[index]
#         print [frequency,Edf,Esf,Erf,Exf,Elf,Etf,Edr,Esr,Err,Exr,Elr,Etr]
#         print Sm[0,0]
        D =(1+(Sm[0,0]-Edf)*(Esf/Erf))*(1+(Sm[1,1]-Edr)*(Esr/Err))-(Sm[0,1]*Sm[1,0]*Elf*Elr)/(Etf*Etr)
#         print D
        S11 =(Sm[0,0]-Edf)/(D*Erf)*(1+(Sm[1,1]-Edr)*(Esr/Err))-(Sm[0,1]*Sm[1,0]*Elf)/(D*Etf*Etr)
        S21 =(Sm[1,0]/(D*Etf))*(1+(Sm[1,1]-Edr)*(Esr-Elf)/Err)
        S12 = (Sm[0,1]/(D*Etr))*(1+(Sm[0,0]-Edf)*(Esf-Elr)/Erf) 
        S22 = (Sm[1,1]-Edr)/(D*Err)*(1+(Sm[0,0]-Edf)*(Esf/Erf))-(Sm[0,1]*Sm[1,0]*Elr)/(D*Etf*Etr)
        s_parameter_out.append([frequency,S11,S21,S12,S22])
    return s_parameter_out

In [8]:
file_path=r'C:\Users\sandersa\PyCharm Projects\Jupyter-Notebooks\StatistiCAL_Files\StatistiCAL_Tests\CalCoefficients.txt'
in_file=open(file_path,'r')
lines=[]
for line in in_file:
    lines.append(map(lambda x:float(x),line.split(" ")))
in_file.close()

TWELVE_TERM_ERROR_COLUMN_NAMES=["Frequency","reEdf","imEdf","reEsf","imEsf",
                                "reErf","imErf","reExf","imExf","reElf","imElf","reEtf","imEtf",
                                "reEdr","imEdr","reEsr","imEsr","reErr","imErr","reExr","imExr",
                                "reElr","imElr","reEtr","imEtr"]
#"row_pattern":make_row_match_string(TWELVE_TERM_ERROR_COLUMN_NAMES),
options={"data_delimiter":" ","row_end_token":'\n'}

cal_coefficients=lines
complex_cal_coefficients=[]
for row in cal_coefficients:
    frequency=[row[0]]
    complex_numbers=row[1:]
    #print np.array(complex_numbers[1::2])
    complex_array=np.array(complex_numbers[0::2])+1.j*np.array(complex_numbers[1::2])
    complex_cal_coefficients.append(frequency+complex_array.tolist())

In [9]:
import pandas
data_frame=pandas.read_csv(file_path)

In [10]:
corrected_s=correct_sparameters(two_port.sparameter_complex,complex_cal_coefficients)

In [13]:
correct_two_port=S2PV1(**{"sparameter_complex":corrected_s,"option_line":"# GHz S RI R 50"})
#correct_two_port.data

In [14]:
two_port.change_data_format("MA")
correct_two_port.change_data_format("MA")
plt.plot(two_port.get_column('Frequency'),two_port.get_column('magS22'),label="Uncorrected")
plt.plot(correct_two_port.get_column('Frequency'),correct_two_port.get_column('magS22'),
         label="Corrected")
plt.legend()
plt.show()

In [15]:
def compare_s2p_plots(list_S2PV1,**options):
    """compare_s2p_plot compares a list of s2p files ploting each on the same axis"""
    defaults={"format":"MA",
              "display_legend":True,
              "save_plot":False,
              "directory":None,
              "specific_descriptor":"Comparision_Plot",
              "general_descriptor":"Plot",
              "file_name":None,
              "labels":None}
    comparision_plot_options={}
    for key,value in defaults.iteritems():
        comparision_plot_options[key]=value
    for key,value in options.iteritems():
        comparision_plot_options[key]=value
    
    # create a set of 6 subplots    
    fig, compare_axes = plt.subplots(nrows=4, ncols=2, figsize=(8,6),dpi=80)
    if comparision_plot_options["labels"] is None:
        labels=[s2p.path for s2p in list_S2PV1]
    else:
        labels=comparision_plot_options["labels"]
    for s2p_index,s2p in enumerate(list_S2PV1):
        # start by changing the format of all the s2p
        s2p.change_data_format(comparision_plot_options["format"])
        column_names=s2p.column_names[1:]
        for index, ax in enumerate(compare_axes.flat):
            #ax.xaxis.set_visible(False)
            if re.search('arg',column_names[index]):
                ax.set_ylabel('Phase(Degrees)',color='green')
            elif re.search('mag',column_names[index]):
                ax.set_ylabel(r'|${\Gamma} $|',color='green')
            ax.set_title(column_names[index])
            # initial plot of
            x=s2p.get_column('Frequency')
            y=np.array(s2p.get_column(column_names[index]))
            ax.plot(x,y,label=labels[s2p_index])
            if comparision_plot_options["display_legend"]:
                ax.legend(loc=1,fontsize='8')
            
    compare_axes.flat[-2].set_xlabel('Frequency(GHz)',color='k')
    compare_axes.flat[-1].set_xlabel('Frequency(GHz)',color='k')
    fig.subplots_adjust(hspace=0)
    plt.tight_layout()
    # Dealing with the save option
    if comparision_plot_options["file_name"] is None:
        file_name=auto_name(specific_descriptor=comparision_plot_options["specific_descriptor"],
                            general_descriptor=comparision_plot_options["general_descriptor"],
                            directory=comparision_plot_options["directory"]
                            ,extension='png',padding=3)
    else:
        file_name=comparision_plot_options["file_name"]
    if comparision_plot_options["save_plot"]:
        #print file_name
        plt.savefig(os.path.join(comparision_plot_options["directory"],file_name))
    else:
        plt.show()
        

In [16]:
compare_s2p_plots([two_port,correct_two_port])

In [17]:
compare_s2p_plots([two_port,correct_two_port],**{"format":"RI","labels":["uncorrected","corrected"]})

In [18]:
compare_s2p_plots([two_port,correct_two_port],format="MA",labels=["uncorrected","corrected"])

In [37]:
compare_s2p_plots([two_port,correct_two_port],**{"format":"DB"})

<img src="./Calibration_Example_Files/Twelve_Term_Correction.png" />
VNA Calibration Coefficients

StatistiCAL stores VNA calibration coefficients in the following order:

frequency  Edf  Esf  Erf  Exf  Elf  Etf  Edr  Esr  Err  Exr  Elr  Etr.

The value of the frequency is in gigahertz. The other values are expressed as real/imaginary pairs.




In [13]:
# Now we need to have a model for the error coefficients and a function that applies them to the s-parameters
class TwelveTermErrorModel(AsciiDataTable):
    """TwevleTermErrorModel holds the error coefficients for a twelve term model. The VNA calibration coefficeients 
    are presumed to be stored in the following order frequency Edf Esf Erf Exf Elf Etf Edr Esr Err Exr Elr Etr, where
    all coefficients are in Real-Imaginary format. """
    pass

    

In [14]:
test_list=[1,2,3,4]

In [15]:
len(test_list)

4

In [16]:
test_list[1::2]

[2, 4]

In [17]:
test_list[0::2]

[1, 3]

In [18]:
np.array(test_list[0::2])+1j*np.array(test_list[1::2])

array([ 1.+2.j,  3.+4.j])